In [1]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import tikzplotlib
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import os
import time
from qiskit.aqua.algorithms import NumPyEigensolver
from typing import List, NamedTuple
from qiskit.chemistry.transformations import FermionicTransformation, FermionicTransformationType, FermionicQubitMappingType
import pylab
import os
import time
from typing import List, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from qiskit import QuantumCircuit
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import PauliExpectation
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.converters import AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator, AerProvider
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import Statevector
from qiskit.test.mock import *
from qiskit.visualization import plot_error_map

/Users/bavithra/miniconda3/envs/qiskit-env/lib/python3.8/site-packages/pyscf/lib/misc.py:47: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
plt.rcParams["figure.dpi"] = 75
plt.style.use("ggplot")

In [3]:
basis_sets = ['sto6g', '631g', '321g']
threshold = 0.00000001

In [16]:
def get_qubit_hamiltonian(d, basis_set, map_type='jordan_wigner'):
    """
    Obtain the qubit operators from the fermionic operators after second quantization.
    Args:
        d: inter-atomic distance
        basis_set: computational chemistry basis set
        map_type: encoding method

    Returns: qubit Hamiltonian as a weighted summed of Paulis, energy shift because of nuclear repulsion, number of particles and number of spin orbitals

    """
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(d) ,
                         unit=UnitsType.ANGSTROM,
                         charge=0,
                         spin=0,
                         basis=basis_set)
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    repulsion_energy = molecule.nuclear_repulsion_energy
    fermionic_op = FermionicOperator(h1,h2)
    qubit_op = fermionic_op.mapping(map_type=map_type, threshold=threshold)
    return qubit_op, repulsion_energy, num_particles, num_spin_orbitals

In [17]:
qubit_ops = {}
for set in basis_sets:
    qubit_ops["qubitOp_{0}".format(set)],_,_,_ = get_qubit_hamiltonian(0.735, set)

for label in qubit_ops:
    print("\n --- {} ---".format(label),qubit_ops[label])



 --- qubitOp_sto6g --- Representation: paulis, qubits: 4, size: 15

 --- qubitOp_631g --- Representation: paulis, qubits: 8, size: 185

 --- qubitOp_321g --- Representation: paulis, qubits: 8, size: 185


In [12]:
class hydrogen_molecule():
    """
    Args class for describing a specific molecular configuration instance"
    """
    def __init__(self, bond_length: float):
        """

        Args:
            bond_length: inter-atomic distance
        """
        self.bond_length = bond_length
        self.mapping = 'jordan_wigner'
        self.basis_set = 'sto6g'

        qubit_ops, repulsion, num_particles, num_spin_orbitals = get_qubit_hamiltonian(self.bond_length, self.basis_set, self.mapping)

        grouper = AbelianGrouper()

        self.qubit_hamiltonian = grouper.convert(qubit_ops.to_opflow())
        self.num_particles = num_particles
        self.num_spin_orbitals = num_spin_orbitals
        self.repulsion = repulsion
        self.num_qubits = self.qubit_hamiltonian.num_qubits

        hamiltonian_matrixform = self.qubit_hamiltonian.to_matrix()
        eigenvalues, _ = np.linalg.eigh(hamiltonian_matrixform)

        self.groundstate_energy = eigenvalues[0]

In [59]:
bond_lengths = np.linspace(0.1, 3.5, 50)

In [13]:
h2_0_735 = hydrogen_molecule(0.735)